In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Model, load_model
from keras.layers import Input, Dense
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Data/PCA_Normalized.csv', index_col = 0)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,BBLE
0,0.535946,-0.340776,-0.378950,-0.409478,0.385434,0.752969,-0.966496,-0.007804,1000010101
1,4.410000,15.830565,7.294441,-3.931946,1.580675,15.990311,4.177770,1.256324,1000010201
2,-0.002913,0.070488,-0.009502,0.081313,-0.012082,-0.091095,-0.047430,0.064557,1000020001
3,0.035085,-0.062495,-0.065342,0.002053,-0.036879,-0.070413,-0.107799,-0.015961,1000020023
4,7.995610,-3.547684,-2.722607,-4.061353,-10.119040,9.130893,-10.615977,-13.862914,1000030001


In [26]:
sum(sorted(df['BBLE'].value_counts()))

1070994

In [4]:
mat = df.iloc[:,:-1].values.astype(np.float32)

In [5]:
mat.shape

(1070994, 8)

In [6]:
input_dim = mat.shape[1]  # 设定输入层
encoding_dim = 4  # 设定编码层

input_layer = Input(shape=(input_dim, ))  # 添加输入层

encoder = Dense(encoding_dim, activation="tanh")(input_layer)  # 编码器输入层使用tanh激活函数

decoder = Dense(input_dim, activation='tanh')(encoder)  # 解码器输入层使用tanh激活函数

autoencoder = Model(inputs=input_layer, outputs=decoder)  # 创建自动编码模型

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
nb_epoch = 3  # 设定循环次数

autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])  # 编译自动编码器

y_predictor = autoencoder.fit(mat, mat, epochs=nb_epoch, shuffle=True)  # 训练模型

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
1070994/1070994 [==============================] - 24s 22us/step - loss: 0.9742 - acc: 0.6596
Epoch 2/3
1070994/1070994 [==============================] - 23s 22us/step - loss: 0.9668 - acc: 0.7021
Epoch 3/3
1070994/1070994 [==============================] - 22s 20us/step - loss: 0.9658 - acc: 0.7084


In [8]:
predictions = autoencoder.predict(mat)

In [9]:
pd.DataFrame(predictions).head()

,0,1,2,3,4,5,6,7
0,0.539081,-0.443383,-0.278159,-0.651390,-0.167545,0.889192,-0.710545,-0.039606
1,0.903240,0.999914,0.988283,-0.233973,0.845411,0.999939,0.967048,0.129222
2,-0.010449,0.058978,0.004460,0.063484,-0.049504,-0.105880,0.003873,0.068871
3,0.025202,-0.076507,-0.056988,-0.037066,-0.069471,-0.059543,-0.127558,0.001882
4,0.994868,-0.994479,-0.868451,-0.999959,-0.803478,0.998039,-0.999994,-0.998309


In [10]:
diff = pd.DataFrame(predictions - mat)

In [11]:
diff.head()

,0,1,2,3,4,5,6,7
0,0.003135,-0.102608,0.100792,-0.241912,-0.552979,0.136223,0.255951,-0.031802
1,-3.506760,-14.830650,-6.306158,3.697973,-0.735264,-14.990372,-3.210722,-1.127102
2,-0.007536,-0.011510,0.013962,-0.017829,-0.037422,-0.014784,0.051303,0.004314
3,-0.009882,-0.014013,0.008354,-0.039119,-0.032592,0.010870,-0.019759,0.017843
4,-7.000741,2.553206,1.854156,3.061394,9.315562,-8.132854,9.615983,12.864605


In [13]:
res = diff.copy()

In [14]:
res['Euclidean_Distance'] = diff.apply(lambda series: series ** 2).apply(sum, axis = 1).apply(lambda series: series ** (1/2))

In [15]:
res['BBLE'] = df['BBLE']

In [16]:
res.head()

,0,1,2,3,4,5,6,7,Euclidean_Distance,BBLE
0,0.003135,-0.102608,0.100792,-0.241912,-0.552979,0.136223,0.255951,-0.031802,0.685626,1000010101
1,-3.506760,-14.830650,-6.306158,3.697973,-0.735264,-14.990372,-3.210722,-1.127102,22.858678,1000010201
2,-0.007536,-0.011510,0.013962,-0.017829,-0.037422,-0.014784,0.051303,0.004314,0.070510,1000020001
3,-0.009882,-0.014013,0.008354,-0.039119,-0.032592,0.010870,-0.019759,0.017843,0.061508,1000020023
4,-7.000741,2.553206,1.854156,3.061394,9.315562,-8.132854,9.615983,12.864605,21.891265,1000030001


In [17]:
res.sort_values('Euclidean_Distance', ascending = False).head(10)

,0,1,2,3,4,5,6,7,Euclidean_Distance,BBLE
632815,-521.087708,352.251160,-751.653564,-274.311157,15.587546,100.409462,-52.850346,-39.277004,1024.940893,4018420001
776305,-136.493774,-620.757385,-22.518629,-488.171234,29.804358,40.768253,372.615570,-214.634872,911.180198,4080100001
565391,-638.978882,-260.656891,372.464752,-63.076454,-38.298172,299.008606,-140.448364,230.709274,884.742928,3085900700
1067359,-75.043800,-414.031250,-223.468826,137.183838,-8.375778,-456.603088,-421.105743,115.928719,803.204341,5078530085
585117,-138.971298,53.677402,134.845490,102.024315,-440.670502,-14.745822,-90.423775,-240.553696,557.886204,4004200001
585438,-120.445244,52.299812,95.866364,95.721092,-400.811615,-127.613327,92.070267,-168.047394,499.246620,4004590005
565397,-285.712921,103.701988,204.508438,132.185974,-33.968765,92.848404,-222.418091,178.572754,492.763180,3085910100
85885,-165.669373,80.689888,61.821171,92.601250,216.507263,-193.002899,211.424393,237.564621,481.265813,1012540010
917941,-100.678345,27.507240,96.208107,50.193512,89.054642,93.793900,-230.519104,-292.688751,422.139731,4142600001
750815,-17.828634,-108.692162,-52.080704,-39.672569,-7.930135,-301.917480,-235.330643,37.218563,405.463862,4066610005E


In [20]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070994 entries, 0 to 1070993
Data columns (total 10 columns):
0                     1070994 non-null float32
1                     1070994 non-null float32
2                     1070994 non-null float32
3                     1070994 non-null float32
4                     1070994 non-null float32
5                     1070994 non-null float32
6                     1070994 non-null float32
7                     1070994 non-null float32
Euclidean_Distance    1070994 non-null float64
BBLE                  1070994 non-null object
dtypes: float32(8), float64(1), object(1)
memory usage: 49.0+ MB


In [19]:
res[['Euclidean_Distance', 'BBLE']].to_csv('Data/Auto Encoder.csv')